In [25]:
import requests
from bs4 import BeautifulSoup
import os.path
import re
allThairestaurantsNYC = []

allThairestaurantsNYC_URL = 'https://www.yelp.com/search?find_desc=thai+restaurants&find_loc=New+York+City&cflt=thai'
r = requests.get(allThairestaurantsNYC_URL)
yelp_text = r.text
soup = BeautifulSoup(yelp_text)

#download the number of counts of restaurants in NYC from the yelp query
restaurant_count = soup.find('span',{'class':'pagination-results-window'}).contents

#use regular expression to extract the digits from the string that was extracted
restaurant_count = re.findall('\d+', str(restaurant_count))

#select the third object in the list which represents the number of restaurants in NYC
restaurant_count = int(restaurant_count[2])

#total number of pages we will need to load. if we do not have an number of reviews divisible by 20, 
#we will need to review an additional page
restaurant_pages = int(restaurant_count/10) 
if restaurant_count%10 > 0:
    restaurant_pages += 1

#creates a list of parameter inputs to load each page of the restaurant 
rest_params_variable = []
for number in range(restaurant_pages):
    number = (number)*10
    rest_params_variable.append({'start' : number })

#loops over each parameter and then apply beautiful soup so we can read the data on each restaurant page

for param in rest_params_variable:
    temp_r = requests.get(allThairestaurantsNYC_URL,params = param)
    temp_text = temp_r.text
    temp_soup = BeautifulSoup(temp_text)

    for each_restaurant in temp_soup.select(".regular-search-result"):
        href_link = each_restaurant.find('a')['href']
        final_link = 'https://www.yelp.com' + href_link
        allThairestaurantsNYC.append(final_link)

#clean the URLs for all ThairestaurantsinNYC of any "?search_key=" jargon
for x in allThairestaurantsNYC:
    index = allThairestaurantsNYC.index(x)
    mystring = x
    keyword = '?'
    before_keyword, keyword, after_keyword = mystring.partition(keyword)
    allThairestaurantsNYC[index] = before_keyword
        
#################################################################################################################


C:\Users\Olarn\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file C:\Users\Olarn\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [129]:
for aThairestaurant in allThairestaurantsNYC: 
#download the reviews on the first page of a restaurant
    restaurant_url = aThairestaurant
    restaurant_name = restaurant_url.replace('https://www.yelp.com/biz/','')
    
    restaurant_reviews_raw = []
    params_variable = []
    html_page_name = []

    r = requests.get(aThairestaurant)
    yelp_text = r.text
    soup = BeautifulSoup(yelp_text)

    #extracts the number of reviews a restaurant has received
    
    if soup.find('span',{'class':'review-count rating-qualifier'}) == None:
        continue
    
    reviews = soup.find('span',{'class':'review-count rating-qualifier'}).contents
    reviews = str(reviews)
    reviews = [int(s) for s in reviews.split() if s.isdigit()]
    if not reviews:
        continue
    review_count = reviews[0]

    #determines the total number of pages we will need to download per restaurant based on number of reviews
    review_pages = int(review_count/20)
    if review_count%20 > 0:
        review_pages += 1

    #download the reviews located on all other yelp pages
    #create a list of request parameter inputs to feed into a FOR loop with requests. Each parameter represents another page.

    #subtract by 1 because range starts at 0 instead of 1, and we would like there to be a total of 13 review pages, not 14
    for number in range(review_pages):
        number = (number)*20
        html_page_name.append(str(number))
        params_variable.append({'start' : number })

    #feed the list into requests to download all webpages
    for param in params_variable:
        temp_r = requests.get(restaurant_url,params = param)
        temp_text = temp_r.text
        temp_soup = BeautifulSoup(temp_text)
        restaurant_reviews_raw.append(temp_soup.encode("utf-8"))

    #save data to an independent html output file
    save_path = r'C:\Users\Olarn\Desktop\Data Science\Springboard\Yelp_Webpages_2'

    for item in restaurant_reviews_raw:
        index = restaurant_reviews_raw.index(item)
        new_path = restaurant_name +'_page_endson_'+ html_page_name[index] + '.html'
        completeName = os.path.join(save_path, new_path)
        thefile = open(completeName, 'wb')
        thefile.write(item)
        thefile.close()

C:\Users\Olarn\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file C:\Users\Olarn\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
